In [41]:
import glob
import pandas as pd
from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings('ignore')

In [42]:
# to get city coordinates
geolocator = Nominatim(user_agent = 'bmartin')

In [43]:
# get all .csv files from path
# files with tweets collected with Twitter API
path = 'data/places/'
files_path = path + '*.csv'
files = glob.glob(files_path)

In [44]:
# empty dataframe
tweets_cities = pd.DataFrame()

# loop over files
for file in files:
    
    # store file in a temporary dataframe
    df = pd.read_csv(file)
    
    # append df into main dataframe
    tweets_cities = tweets_cities.append(df)
    
# reset index    
#tweets_cities = tweets_cities.drop_duplicates(subset = ["geo.place_id"])
tweets_cities = tweets_cities.reset_index(drop = True)
print(tweets_cities.shape)
tweets_cities.head(2)

(119, 4)


,country,full_name,geo.place_id,name
0,Portugal,"Amadora, Portugal",e0649871bd10d9cf,Amadora
1,Venezuela,"Valencia, Venezuela",012af435641420c7,Valencia


In [45]:
tweets_cities['counts']=1

In [46]:
geolocator.geocode("Mérida, Yucatán")

Location(Mérida, Yucatán, México, (20.9670759, -89.6237402, 0.0))

In [47]:
geolocator.geocode("Mérida, Yucatán")[1]

(20.9670759, -89.6237402)

In [48]:
print(geolocator.geocode("Mérida, Yucatán")[1][0]) # lat
print(geolocator.geocode("Mérida, Yucatán")[1][1]) # lng

20.9670759
-89.6237402


In [49]:
def loc(x):
    try:
        return geolocator.geocode(x)[1]
    except:
        return None

In [50]:
tweets_cities["coordinates"] = tweets_cities.full_name.apply(lambda x: loc(x))
tweets_cities.head()

,country,full_name,geo.place_id,name,counts,coordinates
0,Portugal,"Amadora, Portugal",e0649871bd10d9cf,Amadora,1,"(38.7595162, -9.223677009460125)"
1,Venezuela,"Valencia, Venezuela",012af435641420c7,Valencia,1,"(10.170026, -68.0003987)"
2,Brasil,"Río de Janeiro, Brasil",97bcdfca1a2dca59,Río de Janeiro,1,"(-22.9110137, -43.2093727)"
3,Suiza,"Chiasso, Svizzera",a971c3d2181d4305,Chiasso,1,"(45.8355209, 9.0290169)"
4,Guatemala,Guatemala,13d479b108707983,Guatemala,1,"(15.687100699999998, -90.12265453178017)"


In [51]:
tweets_cities.shape

(119, 6)

In [70]:
geoplaces_full_name = tweets_cities.groupby(['geo.place_id','full_name']).counts.sum().reset_index()
geoplaces_full_name

,geo.place_id,full_name,counts
0,0022e3c837579650,"Auckland, New Zealand",1
1,002e24c6736f069d,"Bronx, NY",2
2,0073b76548e5984f,"Sydney, New South Wales",2
3,00905fa63942b3ef,Ottawa - Gatineau,1
4,009924a469d7ace1,"Guayaquil, Ecuador",1
...,...,...,...
102,dee4ad8775ff8102,"Santa Maria, CA",1
103,e0649871bd10d9cf,"Amadora, Portugal",1
104,ea9809774c4145c3,"Mlati, Indonesia",3
105,ebb2c0a78f95fb17,"San Blas Atempa, Oaxaca",1


In [71]:
geoplaces_full_name["coords"] = geoplaces_full_name.full_name.apply(lambda x: loc(x))

In [72]:
geoplaces_full_name=geoplaces_full_name[geoplaces_full_name.coords.isna()==False]

In [73]:
geoplaces_full_name["lat"] = geoplaces_full_name.coords.apply(lambda x: x[0])
geoplaces_full_name["lon"] = geoplaces_full_name.coords.apply(lambda x: x[1])

In [76]:
#geoplaces_full_name.to_csv('geoplaces.csv', index=False)